# Problemas 8: 

Este notebook contiene la solución explicada de 

Dado un grafo no dirigido, implementa un método, has_cycle, que comprueba si el grafo tiene un ciclo o no. Es decir, el método devolverá True si se encuentra algún ciclo, y False eoc. 

Para detectar si hay algún ciclo, debemos recorrer el grafo desde cada vértice. Para ello podemos emplear o bien el recorrido dfs o bien el recorrido bfs. 

Un grafo no dirigido tien un ciclo únicamente si existe un loop (es decir, una arista que conecta un vértice consigo mismo), o bien, un vértice es conectado con alguno de sus antecesores en el recorrido. 




La clase AdjacentVertex representa una tupla donde el primer elemento es un vértice y el segundo el peso asociado. 



In [ ]:
class AdjacentVertex:
    """ This class allows us to represent a tuple
    with an adjacent vertex
    and the weight associated (by default None, for non-unweighted graphs)"""
    def __init__(self, vertex: object, weight: int = 1) -> None:
        self.vertex = vertex
        self.weight = weight

    def __eq_(self, other: 'AdjacentVertex') -> bool:
        if other is None: 
            return False
        return self.vertex == other.vertex and self.weight == other.weight 
        
    def __str__(self) -> str:
        """ returns the tuple (vertex, weight)"""
        if self.weight is not None:
            return '(' + str(self.vertex) + ',' + str(self.weight) + ')'
        else:
            return str(self.vertex)

En la siguiente implementación, prescindimos del atributo directed, porque el grafo siemepre va a ser no dirigido.

In [ ]:
from queue import Queue

class Graph:
    def __init__(self, vertices: list) -> None:
        """ Undirected graph"""
        self._vertices = {}
        for v in vertices:
            self._vertices[v] = []

    def add_vertex(self, vertex: str) -> None:
        if vertex in self._vertices.keys():
            print(vertex, ' already exists!')
            return
        self._vertices[vertex] = []

    def add_edge(self, start: object, end: object, weight: int = 1) -> None:
        if start not in self._vertices.keys():
            print(start, ' does not exist!')
            return
        if end not in self._vertices.keys():
            print(end, ' does not exist!')
            return

        # adds to the end of the list of neighbours for start
        self._vertices[start].append(AdjacentVertex(end, weight))
        # As the graph is undirecgted, we must also add the edge from end to start
        self._vertices[end].append(AdjacentVertex(start, weight))

    def contain_edge(self, start: object, end: object) -> int:
        """ checks if the edge (start, end) exits. It does
        not exist return 0, eoc returns its weight or 1 (for unweighted graphs)"""
        if start not in self._vertices.keys():
            print(start, ' does not exist!')
            return 0
        if end not in self._vertices.keys():
            print(end, ' does not exist!')
            return 0

        # we search the AdjacentVertex whose v is equal to end

        for adj in self._vertices[start]:
            if adj.vertex == end:
                return adj.weight

        return 0  # does not exist

    def remove_edge(self, start: object, end: object):
        """ removes the edge (start, end)"""
        if start not in self._vertices.keys():
            print(start, ' does not exist!')
            return
        if end not in self._vertices.keys():
            print(end, ' does not exist!')
            return

        # we must look for the adjacent AdjacentVertex (neighbour)  whose vertex is end, and then remove it
        exist = False
        for adj in self._vertices[start]:
            if adj.vertex == end:
                exist = True
                self._vertices[start].remove(adj)

        # As the graph is undirected, we must also remove the edge form end to start
        for adj in self._vertices[end]:
            if adj.vertex == start:
                self._vertices[end].remove(adj)

        if not exist: 
            print("({},{}) does not exist!!!!".format(start, end))


    def __str__(self) -> str:
        """ returns a string containing the graph"""
        result = ''
        for v in self._vertices:
            result += '\n'+str(v)+':'
            for adj in self._vertices[v]:
                result += str(adj)+"  "
        result += '\n'
        return result

    def get_adjacent_vertices(self, start: object) -> list:
        """ returns a Python list containing the adjacent
        vertices of vertex. The list only contains the vertices"""
        if start not in self._vertices.keys():
            print(start, ' does not exist!')
            return None
        
        result = []
        for adj in self._vertices[start]:
            result.append(adj.vertex)
        return result




Vamos a implementar el método has_cycle usando el recorrido en profundidad (recursivo)

In [ ]:

class Graph2(Graph):

    def has_cycle_dfs(self) -> bool:
 
        visited = dict.fromkeys(self._vertices.keys(), False)
 
        for v in self._vertices.keys():
            if not visited[v]:
                if self._has_cycle_dfs(v, visited, None):
                    return True
 
        return False
        
    def _has_cycle_dfs(self, v: object, visited: dict, parent: object) -> bool:
        visited[v] = True
        for u in self.get_adjacent_vertices(v):
            if not visited[u]:
                if self._has_cycle_dfs(u, visited, v):
                    return True
            else:
                # If the graph does not have any cycle and u has been already visited, this means that v should come from u
                # Eoc, there is a cycle
                if parent != u:
                    return True
        return False

    def has_cycle_bfs(self) -> bool:
 
        visited = dict.fromkeys(self._vertices.keys(), False)
        parents = dict.fromkeys(self._vertices.keys(), None)

        # get the first vertext that was saved into the graph
        start = list(self._vertices.keys())[0]

        queue=[start]   #we will use a list as a queue
        # while queue is not empy
        while len(queue)>0:
            v = queue.pop(0)    # we get the first element from the queue
            visited[v] = True
            # we gest is adjacent vertices
            for u in self.get_adjacent_vertices(v):
                if not visited[u]:
                    queue.append(u) # we add it to the queue
                    parents[u] = v  # we save its parent
                else:
                    # u has been already visited!!!. In a graph without any cycle, 
                    # this means that there is an edge from u <-> v, that was already visited, that is, parent[v] should be u
                    # Eoc, this means that there is a graph
                    if parents[v] != u:
                        return True
        
        return False
        

Si dado un vértice v, existe un adyacente u, que ya ha sido visitado, pero que su padre es distinto de v, entonces hay un ciclo. 
Si no existe ningún vértice con esas características decimos 

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/3/3d/Undirected_graph.svg/462px-Undirected_graph.svg.png?20060311152740' width='300'>


In [ ]:
labels = [1, 2, 3, 4]
g = Graph2(labels)

g.add_edge(1,2)
g.add_edge(1,3)
g.add_edge(2,3)
g.add_edge(3,4)
# print(g)

print(g)
print(g.has_cycle_dfs())
print(g.has_cycle_bfs())

g.remove_edge(1,3)
print(g)

print(g.has_cycle_dfs())
print(g.has_cycle_bfs())




1:(2,1)  (3,1)  
2:(1,1)  (3,1)  
3:(1,1)  (2,1)  (4,1)  
4:(3,1)  

True
True

1:(2,1)  
2:(1,1)  (3,1)  
3:(2,1)  (4,1)  
4:(3,1)  

False
False
